In [1]:
#Kompresja LZW

#jednocześnie tworzenie słownika i kodowanie

In [2]:
from random import randint
import random as random
from math import log
import bitarray as ba
import operator

In [3]:
def read_file(file):
    return open(file).read()

In [4]:
#słownik dla liczb od 0 - 255
def generateDictionary():
    dict = {}
    for i in range(256):
        dict[bin(i)[2:].zfill(8)] = i
    
    return dict

In [5]:
def read_file_bin(file_name):
    bit_array = ba.bitarray()
    with open(file_name, 'rb') as fh:
        bit_array.fromfile(fh)
    return bit_array

In [6]:
def save_file_bin(file_name, bit_array):
    with open(file_name, 'wb') as fh:
        bit_array.tofile(fh)
    return 1

In [7]:
wikipedia = read_file_bin("norm_wiki_sample.txt")

In [8]:
test = read_file_bin("test.txt")

In [9]:
#Koduje tekst
def encode(text):
    dict = generateDictionary()
    
    numbers = []
    max_dict_val = max(dict.values())
    
    
    
    print("max_dict_val", max_dict_val)
    
    coded_text = ba.bitarray()
    
    p = text[0:8].to01()

    
    
    for i in range(8, len(text), 8):
        #c - obecny ciąg binarny
        c = text[i:i+8].to01()
        if p + c in dict:
            #kod znajduje się w słowniku
            p = p + c
        else:
            #kod nie znajduje się w słowniku
            numbers.append(dict[p])
            dict[p + c] = max_dict_val + 1
            max_dict_val += 1            
            p = c
        
    max_number = max(numbers)
    max_length = len(bin(max_number)[2:])
    print("max_num", max_number)
    print("bin(max_number)", bin(max_number)[2:])
    print("max_length", max_length)
    
    for number in numbers:
        for b in bin(number).zfill(max_length):
            if b == '1':
                coded_text.append(True)
            else:
                coded_text.append(False) 
  
    print("done")
    return coded_text, max_length

In [10]:
#Zapis kodu i zakodowanego tekstu do pliku
def save(text, file_name):
    
    save_file_bin(text, file_name)  
    
    return 0

In [11]:
import struct

In [19]:
#słownik dla liczb od 0 - 255
def generateDictionaryZFill(fill_num):
    dict = {}
    for i in range(256):
        dict[bin(i)[2:].zfill(fill_num)] = bin(i)[2:].zfill(fill_num)
    
    return dict

In [13]:
#Dekoduje tekst
def decode(coded_text, code_len):
    
    dictionary = generateDictionaryZFill(code_len)
    
    max_dict_val = max(dictionary.values())   
    print("max_dict_val", max_dict_val)
    
    #swap key and value
    #dictionary = dict((v,k) for k,v in dictionary.items())
    
    encoded_text = []
    
    #encoded_text = ba.bitarray()
    
    #pierwszy element dodany do słownika
    p = coded_text[0:code_len].to01()
    #p = int(p, 2)
    print("p: ", p)
    max_dict_val += 1 
    dictionary[max_dict_val] = dictionary[p]
    
    
    encoded_text.append(dictionary[p])
    
    for i in range(code_len, len(coded_text), code_len):
        #c - obecny ciąg binarny
        c = coded_text[i:i+code_len].to01()
        #c = int(c, 2)
                
        try:
            test1 = dictionary[c]
        except:
            print("c", c)
            print("max_val", max_dict_val)
            
        dictionary[max_dict_val] = dictionary[max_dict_val] + test1
        encoded_text.append(dictionary[c])        
        max_dict_val += 1 
        dictionary[max_dict_val] = dictionary[c]
        
    
    print("done")
    return encoded_text


In [14]:
def save_file(file_name):
    result = open(file_name, 'w')
    
    for text, num in code.items():
        result.write(text + ";" + str(num) + ";")
    
    result.close()
    return result

In [15]:
#Wczytuje plik binarny
def load(file_name):
    
    file = read_file_bin(file_name)
    
    return file

In [16]:
def compare_strings(original, new_string):
    for org, new in zip(original, new_string):
        if org != new:
            print(org, "!=", new)
            return "Teksty są różne"
    return "Teksty sa równe"

In [17]:
lena = load("lena.bmp")

In [18]:
coded_lena, code_len = encode(lena)

max_dict_val 255
max_num 2683602
bin(max_number) 1010001111001011010010
max_length 22
done


In [24]:
save("coded_lena", coded_lena)

0

In [25]:
coded_lena = load("coded_lena")

In [36]:
decoded_lena = decode(coded_lena, code_len)

max_dict_val 255
p:  66
c 261
max_val 256
c 261
max_val 256
c 262
max_val 256
c 262
max_val 256
c 261
max_val 256
c 268
max_val 256
c 261
max_val 256
c 262
max_val 256
c 261
max_val 256
c 260
max_val 256
c 261
max_val 256
c 281
max_val 256
c 276
max_val 256
c 285
max_val 256
c 262
max_val 256
c 261
max_val 256
c 288
max_val 256
c 290
max_val 256
c 276
max_val 256
c 316
max_val 256
c 286
max_val 256
c 286
max_val 256
c 334
max_val 256
c 337
max_val 256
c 354
max_val 256
c 362
max_val 256
c 367
max_val 256
c 373
max_val 256
c 389
max_val 256
c 391
max_val 256
c 386
max_val 256
c 402
max_val 256
c 415
max_val 256
c 416
max_val 256
c 422
max_val 256
c 434
max_val 256
c 428
max_val 256
c 437
max_val 256
c 447
max_val 256
c 436
max_val 256
c 445
max_val 256
c 370
max_val 256
c 454
max_val 256
c 445
max_val 256
c 461
max_val 256
c 469
max_val 256
c 484
max_val 256
c 474
max_val 256
c 508
max_val 256
c 527
max_val 256
c 529
max_val 256
c 551
max_val 256
c 589
max_val 256
c 602
max_val 256
c 60

c 2434
max_val 256
c 2313
max_val 256
c 2385
max_val 256
c 2419
max_val 256
c 2447
max_val 256
c 2365
max_val 256
c 2454
max_val 256
c 2456
max_val 256
c 2455
max_val 256
c 2402
max_val 256
c 2463
max_val 256
c 2409
max_val 256
c 1906
max_val 256
c 2420
max_val 256
c 2426
max_val 256
c 2423
max_val 256
c 2476
max_val 256
c 1946
max_val 256
c 2420
max_val 256
c 2423
max_val 256
c 2425
max_val 256
c 2432
max_val 256
c 2491
max_val 256
c 2468
max_val 256
c 2509
max_val 256
c 2506
max_val 256
c 2462
max_val 256
c 2464
max_val 256
c 2484
max_val 256
c 2459
max_val 256
c 2461
max_val 256
c 2462
max_val 256
c 2529
max_val 256
c 2440
max_val 256
c 2425
max_val 256
c 2526
max_val 256
c 2461
max_val 256
c 1962
max_val 256
c 2532
max_val 256
c 2543
max_val 256
c 2555
max_val 256
c 2555
max_val 256
c 2541
max_val 256
c 2482
max_val 256
c 2517
max_val 256
c 720
max_val 256
c 2512
max_val 256
c 2574
max_val 256
c 1973
max_val 256
c 1967
max_val 256
c 2542
max_val 256
c 2585
max_val 256
c 2583
max_va

c 2054
max_val 256
c 3976
max_val 256
c 3803
max_val 256
c 2029
max_val 256
c 3984
max_val 256
c 2027
max_val 256
c 3815
max_val 256
c 2033
max_val 256
c 2026
max_val 256
c 1994
max_val 256
c 1996
max_val 256
c 1998
max_val 256
c 1912
max_val 256
c 2540
max_val 256
c 2491
max_val 256
c 2414
max_val 256
c 2434
max_val 256
c 1943
max_val 256
c 1934
max_val 256
c 1936
max_val 256
c 1924
max_val 256
c 2373
max_val 256
c 2361
max_val 256
c 351
max_val 256
c 2318
max_val 256
c 4015
max_val 256
c 4011
max_val 256
c 4019
max_val 256
c 4017
max_val 256
c 342
max_val 256
c 1893
max_val 256
c 990
max_val 256
c 348
max_val 256
c 4032
max_val 256
c 995
max_val 256
c 444
max_val 256
c 1891
max_val 256
c 1893
max_val 256
c 1895
max_val 256
c 348
max_val 256
c 341
max_val 256
c 4043
max_val 256
c 4035
max_val 256
c 453
max_val 256
c 444
max_val 256
c 4035
max_val 256
c 4030
max_val 256
c 4051
max_val 256
c 1907
max_val 256
c 4028
max_val 256
c 4023
max_val 256
c 2235
max_val 256
c 4009
max_val 256
c 1

max_val 256
c 959
max_val 256
c 4051
max_val 256
c 349
max_val 256
c 1180
max_val 256
c 4308
max_val 256
c 970
max_val 256
c 972
max_val 256
c 4311
max_val 256
c 4422
max_val 256
c 982
max_val 256
c 1159
max_val 256
c 4804
max_val 256
c 4253
max_val 256
c 962
max_val 256
c 5023
max_val 256
c 4580
max_val 256
c 1042
max_val 256
c 1837
max_val 256
c 5033
max_val 256
c 1008
max_val 256
c 1010
max_val 256
c 1733
max_val 256
c 1013
max_val 256
c 1015
max_val 256
c 1014
max_val 256
c 363
max_val 256
c 5044
max_val 256
c 4566
max_val 256
c 398
max_val 256
c 1066
max_val 256
c 1030
max_val 256
c 1374
max_val 256
c 1862
max_val 256
c 1230
max_val 256
c 493
max_val 256
c 4578
max_val 256
c 1837
max_val 256
c 1838
max_val 256
c 4343
max_val 256
c 1680
max_val 256
c 1722
max_val 256
c 1063
max_val 256
c 1055
max_val 256
c 1420
max_val 256
c 4337
max_val 256
c 1070
max_val 256
c 4317
max_val 256
c 1068
max_val 256
c 4335
max_val 256
c 1677
max_val 256
c 1242
max_val 256
c 1769
max_val 256
c 4341
ma

max_val 256
c 2024
max_val 256
c 2036
max_val 256
c 2033
max_val 256
c 3815
max_val 256
c 5794
max_val 256
c 5796
max_val 256
c 5793
max_val 256
c 5800
max_val 256
c 2072
max_val 256
c 5806
max_val 256
c 2054
max_val 256
c 3979
max_val 256
c 2103
max_val 256
c 2095
max_val 256
c 2074
max_val 256
c 2058
max_val 256
c 5823
max_val 256
c 2113
max_val 256
c 2094
max_val 256
c 2093
max_val 256
c 2095
max_val 256
c 2091
max_val 256
c 2100
max_val 256
c 2086
max_val 256
c 5838
max_val 256
c 2099
max_val 256
c 5839
max_val 256
c 2083
max_val 256
c 2112
max_val 256
c 2112
max_val 256
c 2106
max_val 256
c 2114
max_val 256
c 2116
max_val 256
c 2118
max_val 256
c 2171
max_val 256
c 5850
max_val 256
c 2120
max_val 256
c 2122
max_val 256
c 2124
max_val 256
c 2129
max_val 256
c 2123
max_val 256
c 2125
max_val 256
c 2127
max_val 256
c 2273
max_val 256
c 2168
max_val 256
c 2132
max_val 256
c 2225
max_val 256
c 2200
max_val 256
c 2250
max_val 256
c 5864
max_val 256
c 2132
max_val 256
c 2257
max_val 256


...

max_val 256
c 8835
max_val 256
c 16529
max_val 256
c 3961
max_val 256
c 8863
max_val 256
c 5903
max_val 256
c 11483
max_val 256
c 16534
max_val 256
c 5892
max_val 256
c 14015
max_val 256
c 8864
max_val 256
c 14015
max_val 256
c 11524
max_val 256
c 14050
max_val 256
c 8887
max_val 256
c 5931
max_val 256
c 11501
max_val 256
c 14016
max_val 256
c 11522
max_val 256
c 11535
max_val 256
c 5935
max_val 256
c 2250
max_val 256
c 8869
max_val 256
c 8848
max_val 256
c 14039
max_val 256
c 8832
max_val 256
c 5953
max_val 256
c 5860
max_val 256
c 14041
max_val 256
c 5896
max_val 256
c 2195
max_val 256
c 8888
max_val 256
c 8869
max_val 256
c 2129
max_val 256
c 8845
max_val 256
c 11499
max_val 256
c 8885
max_val 256
c 2227
max_val 256
c 5918
max_val 256
c 2243
max_val 256
c 11526
max_val 256
c 5934
max_val 256
c 16550
max_val 256
c 11503
max_val 256
c 2238
max_val 256
c 2293
max_val 256
c 14063
max_val 256
c 11545
max_val 256
c 14056
max_val 256
c 14056
max_val 256
c 11556
max_val 256
c 14863
max_val 

max_val 256
c 9620
max_val 256
c 12212
max_val 256
c 17267
max_val 256
c 12195
max_val 256
c 9587
max_val 256
c 14737
max_val 256
c 17224
max_val 256
c 3399
max_val 256
c 12167
max_val 256
c 12166
max_val 256
c 9652
max_val 256
c 12164
max_val 256
c 14686
max_val 256
c 6568
max_val 256
c 4504
max_val 256
c 15255
max_val 256
c 4462
max_val 256
c 701
max_val 256
c 2865
max_val 256
c 14816
max_val 256
c 5351
max_val 256
c 3605
max_val 256
c 14821
max_val 256
c 3568
max_val 256
c 14825
max_val 256
c 14825
max_val 256
c 957
max_val 256
c 13493
max_val 256
c 3601
max_val 256
c 5021
max_val 256
c 3576
max_val 256
c 2830
max_val 256
c 9688
max_val 256
c 4801
max_val 256
c 319
max_val 256
c 10328
max_val 256
c 10328
max_val 256
c 9751
max_val 256
c 7585
max_val 256
c 14843
max_val 256
c 4496
max_val 256
c 2175
max_val 256
c 2339
max_val 256
c 17333
max_val 256
c 3688
max_val 256
c 14864
max_val 256
c 9951
max_val 256
c 9951
max_val 256
c 6940
max_val 256
c 4531
max_val 256
c 2216
max_val 256
c 

c 15435
max_val 256
c 4467
max_val 256
c 7575
max_val 256
c 15440
max_val 256
c 7581
max_val 256
c 4469
max_val 256
c 10326
max_val 256
c 15444
max_val 256
c 15450
max_val 256
c 12910
max_val 256
c 12906
max_val 256
c 12906
max_val 256
c 4218
max_val 256
c 12905
max_val 256
c 7584
max_val 256
c 10328
max_val 256
c 12906
max_val 256
c 7592
max_val 256
c 15461
max_val 256
c 4979
max_val 256
c 12922
max_val 256
c 10368
max_val 256
c 15476
max_val 256
c 4804
max_val 256
c 7541
max_val 256
c 716
max_val 256
c 12933
max_val 256
c 15469
max_val 256
c 12931
max_val 256
c 15473
max_val 256
c 4534
max_val 256
c 10368
max_val 256
c 5247
max_val 256
c 988
max_val 256
c 369
max_val 256
c 4431
max_val 256
c 375
max_val 256
c 1041
max_val 256
c 1085
max_val 256
c 11006
max_val 256
c 400
max_val 256
c 5666
max_val 256
c 467
max_val 256
c 5061
max_val 256
c 12953
max_val 256
c 1454
max_val 256
c 5672
max_val 256
c 5663
max_val 256
c 1180
max_val 256
c 5082
max_val 256
c 15870
max_val 256
c 1277
max_val

c 1837
max_val 256
c 1725
max_val 256
c 16330
max_val 256
c 1302
max_val 256
c 1745
max_val 256
c 1158
max_val 256
c 13517
max_val 256
c 1306
max_val 256
c 1131
max_val 256
c 8211
max_val 256
c 15930
max_val 256
c 7292
max_val 256
c 1577
max_val 256
c 16092
max_val 256
c 16190
max_val 256
c 4639
max_val 256
c 7605
max_val 256
c 5435
max_val 256
c 8422
max_val 256
c 1332
max_val 256
c 5366
max_val 256
c 5359
max_val 256
c 3594
max_val 256
c 15572
max_val 256
c 7607
max_val 256
c 13637
max_val 256
c 1106
max_val 256
c 8185
max_val 256
c 1356
max_val 256
c 392
max_val 256
c 16118
max_val 256
c 8276
max_val 256
c 5192
max_val 256
c 1300
max_val 256
c 13502
max_val 256
c 10835
max_val 256
c 11272
max_val 256
c 11289
max_val 256
c 1032
max_val 256
c 5389
max_val 256
c 11238
max_val 256
c 4576
max_val 256
c 463
max_val 256
c 436
max_val 256
c 1755
max_val 256
c 1755
max_val 256
c 5097
max_val 256
c 1277
max_val 256
c 7450
max_val 256
c 10209
max_val 256
c 1871
max_val 256
c 16197
max_val 256


KeyboardInterrupt: 

In [39]:
save("lena2.bmp", coded_lena)

0

In [37]:
compare_strings(lena, coded_lena)

True != False


'Teksty są różne'

0

In [37]:
decoded = decode(coded, code)

In [38]:
compare_strings(wikipedia, decoded)

'Teksty sa równe'

In [40]:
length = 0
i = 0

for char, c in code.items():
    i += chars[char]
    length += len(c) * chars[char]
    

oczekiwana_dlugosc_kodu = length / i

In [41]:
oczekiwana_dlugosc_kodu

4.3090155002237935

In [42]:
efektywnosc_kodowania2 = entropia_wikipedii / oczekiwana_dlugosc_kodu

In [43]:
efektywnosc_kodowania2

0.9933582848516693